任务3.1

In [ ]:
import numpy as np
import pandas as pd

dataX1 =pd.read_csv('数据/financial_data.csv').iloc[:,2:17]

datay = np.array(pd.read_csv('数据/financial_data.csv').iloc[:,17]).reshape(-1,1)

dataX1.columns


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] =False #减号unicode编码


datac = pd.read_csv('数据/financial_data.csv').iloc[:,2:18]
corr =datac.corr()

ax = plt.subplots(figsize=(16, 12))#调整画布大小
ax = sns.heatmap(corr,vmax=1, cmap='rainbow', annot=True)#画热力图 annot=True 表示显示系数
# 设置刻度字体大小
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)

plt.title('特征热力图')
plt.show()

任务3.2

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import lightgbm as lgb



dataX = np.array(dataX1[[ 'ZCJZSS','YYSR','YYCB', 'YYSJJFJ', 'YWFY' ]])
"""训练集 验证集构建"""
X_train, X_test, y_train, y_test = train_test_split(dataX, datay, test_size=0.2,random_state=42)


In [ ]:
"""模型训练"""
model = LGBMRegressor()

# r2_score:0.886290371005902
# mse:8.338757275893952

# 给定参数
# model = LGBMRegressor(boosting='gbdt',  # gbdt \ dart
#                       n_estimators=300,  # 迭代次数
#                       learning_rate=0.1,  # 步长
#                       max_depth=10,  # 树的最大深度
#                       seed=42,  # 指定随机种子，为了复现结果
#                       )

# r2_score:0.9043019057586194
# mse:7.017903291953562

model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
# print(y_pred)

for m, n in zip(y_pred, y_test):
    if m / n - 1 > 0.1:
        print('预测值为{0}, 真是结果为{1}, 预测结果偏差大于10%'.format(m, n))


def metrics_sklearn(y_valid, y_pred_):
    """模型效果评估"""
    r2 = r2_score(y_valid, y_pred_)
    print('r2_score:{0}'.format(r2))

    mse = mean_squared_error(y_valid, y_pred_)
    print('mse:{0}'.format(mse))


"""模型效果评估"""
metrics_sklearn(y_test, y_pred)

def adj_params():
    """模型调参"""
    params = {
        'n_estimators': [100, 200, 300, 400],
        # 'learning_rate': [0.01, 0.03, 0.05, 0.1],
        'max_depth': [5, 8, 10, 12]
    }

    other_params = {'learning_rate': 0.1, 'seed': 42}
    model_adj = LGBMRegressor(**other_params)

    # sklearn提供的调参工具，训练集k折交叉验证(消除数据切分产生数据分布不均匀的影响)
    optimized_param = GridSearchCV(estimator=model_adj, param_grid=params, scoring='r2', cv=5, verbose=1)
    # 模型训练
    optimized_param.fit(X_train, y_train)

    # 对应参数的k折交叉验证平均得分
    means = optimized_param.cv_results_['mean_test_score']
    params = optimized_param.cv_results_['params']
    for mean, param in zip(means, params):
        print("mean_score: %f,  params: %r" % (mean, param))
    # 最佳模型参数
    print('参数的最佳取值：{0}'.format(optimized_param.best_params_))
    # 最佳参数模型得分
    print('最佳模型得分:{0}'.format(optimized_param.best_score_))


adj_params()

In [ ]:
params = {
        'n_estimators': [100, 200, 300, 400],
        # 'learning_rate': [0.01, 0.03, 0.05, 0.1],
        'max_depth': [5, 8, 10, 12]
    }

other_params = {'learning_rate': 0.1, 'seed': 42}



model = LGBMRegressor(boosting='gbdt',  # gbdt \ dart
                      n_estimators=200,  # 迭代次数
                      learning_rate=0.1,  # 步长
                      max_depth=5,  # 树的最大深度
                      seed=42,  # 指定随机种子，为了复现结果
                      )

model.fit(X_train, y_train)

In [ ]:
"""模型保存"""
model.booster_.save_model('financial.txt')

"""模型加载"""
rgs = lgb.Booster(model_file='financial.txt')

"""模型参数打印"""
print('模型参数值-开始'.center(20, '='))
# lightgbm模型参数直接打开模型文件查看更为方便
model_params = rgs.dump_model()
print(model_params)
print('模型参数值-结束'.center(20, '='))

"""预测验证数据"""
y_pred = rgs.predict(X_test)

"""模型效果评估"""
metrics_sklearn(y_test, y_pred)

In [ ]:
import pandas as pd
testdata = pd.read_csv('数据/test.csv')
X_test_reality = testdata[[ 'LDBL','YYSR','YYCB', 'YYSJJFJ', 'YWFY' ]]
y_pred_reality = rgs.predict(X_test_reality)
testdata['LRZE'] =y_pred_reality

testdata.to_csv('test.csv', encoding='utf-8', index=False)

In [ ]:
import pandas as pd
testdata = pd.read_csv('数据/test.csv')
X_test_reality = testdata[[ 'LDBL','YYSR','YYCB', 'YYSJJFJ', 'YWFY' ]]
y_pred_reality = rgs.predict(X_test_reality)
testdata['LRZE'] =y_pred_reality

testdata.to_csv('test.csv')
y_pred_reality

In [ ]:
from sklearn.ensemble import RandomForestRegressor

dataX2 = pd.read_csv('数据/financial_data.csv')[['LDBL','ZCFZL','CHZZL','ZCBCL','YSZKZZL','FLAG']]
secX = dataX2.iloc[:,:5]
sexy = dataX2.iloc[:,5]


rf = RandomForestRegressor() # 这里使用了默认的参数设置
rf.fit(secX, sexy) # 进行模型的训练
# 预测样本
instance0 = pd.read_csv('数据/financial_data_new.csv')[['TICKER_SYMBOL','LDBL','ZCFZL','CHZZL','ZCBCL','YSZKZZL']]

instance=instance0.iloc[:,1:]
instance0['flag'] = rf.predict(instance)
print(instance0.loc[instance0['flag']==1.0])




In [ ]:
dataXweak = dataX2.loc[dataX2['FLAG']==1]
corr1 =dataXweak[['LDBL','ZCFZL','CHZZL','ZCBCL','YSZKZZL']].corr()
corr1

In [ ]:
import pandas as pd
dataX2 = pd.read_csv('数据/financial_data.csv')[['LDBL','ZCFZL','CHZZL','ZCBCL','YSZKZZL','FLAG']]
dataXweak = dataX2.loc[dataX2['FLAG']==1]

In [ ]:

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import copy

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler


def process_features(X):
       scaler = StandardScaler()
       X = scaler.fit_transform(X)
       m, n = X.shape
       X = np.c_[np.ones((m, 1)), X]
       return X


def enteropy(data0):
       label_need1 = data0.keys()

       data1 = data0[label_need1].values
       data1 = process_features(data1)

       # 指标正向    化处理后数据为data12
       data12 = data1[:, 1:]

       # 越小越优指标位置,注意python是从0开始计数，对应位置也要相应减1
       index = [2]
       for i in range(0, len(index)):
              data12[:, index[i]] = max(data1[:, index[i]]) - data1[:, index[i]]

       # 某点最优型指标
       index1 = [2]
       a = 90  # 最优型数值
       for i in range(0, len(index1)):
              data12[:, index1[i]] = 1 - abs(data1[:, index1[i]] - a) / max(abs(data1[:, index1[i]] - a))

       # 0.002~1区间归一化
       [m, n] = data12.shape
       data3 = copy.deepcopy(data12)
       ymin = 0.002
       ymax = 1
       for j in range(0, n):
              d_max = max(data12[:, j])
              d_min = min(data12[:, j])
              data3[:, j] = (ymax - ymin) * (data12[:, j] - d_min) / (d_max - d_min) + ymin

       # 计算信息熵

       p = copy.deepcopy(data3)
       for j in range(0, n):
              p[:, j] = data3[:, j] / sum(data3[:, j])

       E = copy.deepcopy(data3[0, :])
       for j in range(0, n):
              E[j] = -1 / np.log(m) * sum(p[:, j] * np.log(p[:, j]))

       w = (1 - E) / sum(1 - E)
       w = np.array(w).reshape(-1, 1)
       return w


dataXweaksp = dataXweak[['ZCFZL', 'CHZZL', 'ZCBCL']]
zdata = np.dot(np.array(dataXweaksp), enteropy(dataXweaksp))
ydata = np.array(dataXweak[['YSZKZZL']])
xdata = np.array(dataXweak[['LDBL']])
xyz = np.hstack((np.hstack((xdata, ydata)), np.hstack((zdata, zdata))))[:, :3]
xyz
maxabs = MaxAbsScaler()
# 标准化处理
xyz = maxabs.fit_transform(xyz)

# 绘图
x = xyz[:, 0]
y = xyz[:, 1]
z = xyz[:, 2]
plt.rcParams['font.sans-serif'] = ['SimHei']
fig = plt.figure(figsize=(16, 9))  # 创建一个画布figure，然后在这个画布上加各种元素。
ax = Axes3D(fig)  # 将画布作用于 Axes3D 对象上。
fig.add_axes(ax)
ax.scatter(x, y, z, c='b')
# 添加坐标轴
ax.set_xlabel('LDBL', fontdict={'size': 10, 'color': 'black'})
ax.set_ylabel('YSZKZZL', fontdict={'size': 10, 'color': 'black'})
ax.set_zlabel('聚合参数', fontdict={'size': 10, 'color': 'black'})
plt.show()


In [ ]:
sde = dataXweak[['LDBL','ZCFZL','CHZZL','ZCBCL','YSZKZZL']]
jihe = np.mean(np.array(sde),axis=0)
jihe

In [ ]:

tesd = pd.read_csv('数据/financial_data_new.csv')[['TICKER_SYMBOL','LDBL','ZCFZL','CHZZL','ZCBCL','YSZKZZL']]
tesd1 = np.array(tesd[['LDBL','ZCFZL','CHZZL','ZCBCL','YSZKZZL']])
tuf = []
for i in range(5):

       tufi =(tesd1[i][0]-jihe[0])**2 + (tesd1[i][1]-jihe[1])**2 + (tesd1[i][2]-jihe[2])**2 + (tesd1[i][3]-jihe[3])**2+(tesd1[i][4] - jihe[4])**2
       tuf.append(tufi)
tuf = pd.DataFrame(tuf)
tesd = pd.concat([tesd,tuf],axis=1)

print(tesd[tesd[0]==tesd[0].min()])

